In [1]:
try:
    import google.colab, sys
    !git clone https://github.com/mualal/hydrofracturing.git
    sys.path.append('hydrofracturing')
    %cd hydrofracturing
except:
    import sys
    sys.path.append('..')

In [2]:
import numpy as np
from sympy import *

# Распределение потоков между трещинами

<center> <img src="../images/flow_distribution_between_fractures_1.jpg" width="700px" /> </center>

<center> <img src="../images/flow_distribution_between_fractures_2.jpg" width="700px" /> </center>

## Законы Кирхгофа

Весь расход, который закачиваем в скважину, перераспределяется между трещинами (первый закон Кирхгофа):
$$
Q_0=\sum_{i=1}^{N}{Q_i}
$$

Можно независимо рассматривать каждый из путей (к каждой из трещин) и считать гидродинамические сопротивления независимо (второй закон Кирхгофа):
$$
p_0=\sigma_{min,i}+p_{net,i}+\Delta p_{perf,i}-\sum_{j=1}^{i}{\Delta p_{h,j}}+\sum_{j=1}^{i}\Delta p_{fric,j},
$$
где

$\sigma_{min,i}$ -- давление закрытия (минимальное напряжение в пласте) на $i$-ой трещине;

$p_{net,i}=p_{frac,i}-\sigma_{min,i}$ -- давление на $i$-ой трещине (из модели трещины);

$\Delta p_{perf,i}$ -- падение давления вдоль перфорации $i$-ой трещины;

$\Delta p_{h,i}$ -- падение гидростатического давления между $i$-ой и $(i-1)$-ой трещинами;

$\Delta p_{fric,i}$ -- падение давления на трение между $i$-ой и $(i-1)$-ой трещинами.


В итоге, получаем следующую систему уравнений:
$$
\begin{cases}
Q_0=\sum\limits_{i=1}^{N}{Q_i}\\
p_0=\sigma_{min,i}+p_{net,i}(Q_i)+\Delta p_{perf,i}(Q_i)-\sum\limits_{j=1}^{i}{\Delta p_{h,j}}+\sum\limits_{j=1}^{i}\Delta p_{fric,j}(Q_i)
\end{cases}
$$
относительно неизвестных расходов $Q_i$ и забойного давления $p_0$.

## Формула для давления $p_{net,i}$ на $i$-ой трещине

Аналитический подход (с использованием PKN модели):
$$
p_{net,i}(Q_i)=a_iQ_i^{\frac{n}{2n+3}}V_i^{\frac{1}{2n+3}},
$$
где $a_i=\left(\dfrac{(n+3)(2n+1)^n \cdot K\cdot (E_i')^{2n+2}}{\pi\, 2^{2n}n^n\phi^n h_i^{3n+3}}\right)^{\!\frac{1}{2n+3}}$ -- параметр жёсткости,


$Q_i$ и $V_i$ -- расход на $i$-ой трещине и объём $i$-ой трещины;

$K$ и $n$ -- реологические параметры степенной (неньютоновской) жидкости;

$E_i'$ -- модуль плоской деформации $i$-ой трещины;

$\phi$ -- геометрический параметр;

$h_i$ -- мощность продуктивной зоны.



In [3]:
frac_count = 3  # количество трещин

Q_0, p_0 = symbols('Q0 p0', real=True)  # расход и давление на забое


Q = np.array(symbols(f'Q1:{frac_count+1}', real=True))  # расходы на каждой из трещин
V = np.array(symbols(f'V1:{frac_count+1}', real=True))  # объём каждой из трещин
E = np.array(symbols(f'E1:{frac_count+1}', real=True))  # модуль плоской деформации каждой из трещин
h = np.array(symbols(f'h1:{frac_count+1}', real=True))  # мощность продуктивной зоны каждой из трещин
K, n, phi = symbols('K n phi', real=True)  # реологические параметры жидкости и геометрический параметр трещины

a = (((n + 3) * (2 * n + 1)**n * K * E**(2*n+2)) / (S.Pi * 2**(2 * n) * n**n * phi**n * h**(3 * n + 3))) ** (1 / (2 * n + 3))
p_net = a * Q**(n/(2*n+3)) * V**(1/(2*n+3))  # давление на каждой из трещин
#[diff(ell, Q[i]) for i,ell in enumerate(p_net)]
print('Давление на 1-ой трещине:')
display(p_net[0])

Давление на 1-ой трещине:


Q1**(n/(2*n + 3))*V1**(1/(2*n + 3))*(E1**(2*n + 2)*K*h1**(-3*n - 3)*(n + 3)*(2*n + 1)**n/(2**(2*n)*pi*n**n*phi**n))**(1/(2*n + 3))

## Формула для падения давления $\text{Δ} p_{perf,i}$ на перфорациях

Эмпирическая формула для падения давления на перфорациях:
$$
\Delta p_{perf,i}=\frac{8\rho_s}{\pi^2 C_{d,i}^2 n_{p,i}^2 d_{p_i}^4}Q_i\left|Q_i\right|,
$$
где $\rho_s$ -- средняя плотность смеси;

$n_{p,i}, d_{p,i}$ -- количество и диаметр перфораций;


$C_{d,i}=\dfrac{\text{min}(d_{jet})}{d_p}$ -- безразмерный коэффициент эррозии (в случае отсутствия твёрдых частичек в потоке $C_{d,i}\in\left[0.5,0.6\right]$, а с твёрдыми частичками в потоке $C_{d,j}\in\left[0.6,0.95\right]$  из-за эррозии перфорации).


Зависимость плотности от концентрации?

In [4]:
n_p = np.array(symbols(f'n_p1:{frac_count+1}', real=True))  # количество перфораций на каждой из трещин
d_p = np.array(symbols(f'd_p1:{frac_count+1}', real=True))  # диаметр перфораций на каждой из трещин
C_d = np.array(symbols(f'C_d1:{frac_count+1}', real=True))  # коэффициент эррозии на каждой из трещин
rho_s = symbols('rho_s', real=True)  # средняя плотность смеси

delta_p_perf = 8 * rho_s / (S.Pi**2 * C_d**2 * n_p**2 * d_p**4) * Q * abs(Q)  # падение давления на перфорациях на каждой из трещин
print(f'Падение давления на перфорациях 1-ой трещины:')
display(delta_p_perf[0])

Падение давления на перфорациях 1-ой трещины:


8*Q1*rho_s*Abs(Q1)/(pi**2*C_d1**2*d_p1**4*n_p1**2)

## Формула для гидростатического давления $\text{Δ} p_{h,i}$

Гидростатическое давление на каждом интервале рассчитывается по следующей формуле:
$$
\Delta p_{h,i}(t,x)=\int\limits_{x_{i-1}}^{x_i}{\rho(c(t,s))\cdot g\cdot \sin{(\theta(s))}ds},
$$
где $x_i$ -- измеренная глубина (MD) $i$-ой трещины;

$\rho(c(t,s))$ -- плотность смеси, которая зависит от динамически меняющейся концентрации проппанта;

$g$ -- ускорение свободного падения;

$\theta(s)$ -- угол между скважиной и поверхностью Земли в данной точке.

## Формула для падения давления $\text{Δ} p_{fric,i}$ на трение

Падение давления на трение на каждом интервале рассчитывается по следующей формуле:
$$
\Delta p_{fric,i}=\int\limits_{x_{i-1}}^{x_i}{f\frac{\rho u_{m,i}^2}{R_i}}=\int\limits_{x_{i-1}}^{x_i}{\frac{\rho(c(t,s))\cdot f(Re)\cdot \left(Q_0-\sum\limits_{j=1}^{i-1}{Q_j}\right)^{\!2}}{R_i(s)S_i^2(s)}}ds,
$$
где $f=\dfrac{\tau}{\rho u_{m,i}^2/2}=\dfrac{16}{Re}=\dfrac{16\mu_s}{\rho u_m(2R)}$ -- коэффициент трения Фаннинга;

$\rho(c(t,s))$ -- плотность смеси, которая зависит от динамически меняющейся концентрации проппанта;

$u_{m,i}=\dfrac{Q_0-\sum\limits_{j=1}^{i-1}{Q_j}}{S_i}$ -- средняя скорость на рассматриваемом участке трубы;

$S_i$ -- площадь сечения рассматриваемого участка трубы;

$R_i$ -- радиус рассматриваемого участка трубы;

$Re=\dfrac{\rho u_m(2R)}{\mu_s}$ -- число Рейнольдса.

Зависимость площади сечения от координаты? Зависимость плотности от концентрации?

In [5]:
R = np.array(symbols(f'R1:{frac_count+1}', real=True))  # радиус участков трубы к каждой из трещин
L = np.array(symbols(f'L1:{frac_count+1}', real=True))  # длина участков трубы, ведущих к каждой из трещин

u_m = np.array(symbols(f'u_m1:{frac_count+1}', real=True))  # средняя скорость на рассматриваемом участке к каждой из трещин
for i, u in enumerate(u_m):
    u_m[i] = (Q_0 - sum([Q[j] for j in range(i)])) / (S.Pi * R[i]**2)

delta_p_fric = 8 * (K / R) * (u_m / R) * L # падение давления на трение в трубе для каждой из трещин
print('Педение давления на трение в трубе между забоем и 1-ой трещиной:')
display(delta_p_fric[0])

Педение давления на трение в трубе между забоем и 1-ой трещиной:


8*K*L1*Q0/(pi*R1**4)

## Все полученные из законов Кирхгофа уравнения

In [6]:
all_eqns = np.array([])  # массив со всеми уравнениями законов Кирхгофа
sigma_min = symbols(f'sigma_min1:{frac_count+1}')  # давление закрытия для каждой из трещин

# добавляем уравнения второго закона Кирхгофа в массив all_eqns
for i in range(frac_count):
    all_eqns = np.append(all_eqns, p_0 - sigma_min[i] - p_net[i] - delta_p_perf[i] - sum([delta_p_fric[j] for j in range(i+1)]))

# добавляем уравнение первого закона Кирхгофа в массив all_eqns
all_eqns = np.append(all_eqns, Q_0 - sum(Q))

<center> <img src="../images/flow_distribution_scheme.jpg" width="700px" /> </center>

In [7]:
print('Все уравнения законов Кирхгофа: ')
for eqn in all_eqns:
    display(eqn)

Все уравнения законов Кирхгофа: 


-8*K*L1*Q0/(pi*R1**4) - Q1**(n/(2*n + 3))*V1**(1/(2*n + 3))*(E1**(2*n + 2)*K*h1**(-3*n - 3)*(n + 3)*(2*n + 1)**n/(2**(2*n)*pi*n**n*phi**n))**(1/(2*n + 3)) + p0 - sigma_min1 - 8*Q1*rho_s*Abs(Q1)/(pi**2*C_d1**2*d_p1**4*n_p1**2)

-8*K*L1*Q0/(pi*R1**4) - 8*K*L2*(Q0 - Q1)/(pi*R2**4) - Q2**(n/(2*n + 3))*V2**(1/(2*n + 3))*(E2**(2*n + 2)*K*h2**(-3*n - 3)*(n + 3)*(2*n + 1)**n/(2**(2*n)*pi*n**n*phi**n))**(1/(2*n + 3)) + p0 - sigma_min2 - 8*Q2*rho_s*Abs(Q2)/(pi**2*C_d2**2*d_p2**4*n_p2**2)

-8*K*L1*Q0/(pi*R1**4) - 8*K*L2*(Q0 - Q1)/(pi*R2**4) - 8*K*L3*(Q0 - Q1 - Q2)/(pi*R3**4) - Q3**(n/(2*n + 3))*V3**(1/(2*n + 3))*(E3**(2*n + 2)*K*h3**(-3*n - 3)*(n + 3)*(2*n + 1)**n/(2**(2*n)*pi*n**n*phi**n))**(1/(2*n + 3)) + p0 - sigma_min3 - 8*Q3*rho_s*Abs(Q3)/(pi**2*C_d3**2*d_p3**4*n_p3**2)

Q0 - Q1 - Q2 - Q3

## Итеративная процедура решения с помощью метода Ньютона

In [8]:
# матрица Якоби
J = np.zeros(shape=(frac_count+1, frac_count+1), dtype=object)
for i, _ in enumerate(all_eqns):
    for j, _ in enumerate(Q):
        J[i, j] = diff(all_eqns[i], Q[j])
    J[i, -1] = diff(all_eqns[i], p_0)

# запись матрицы Якоби в символьном виде
J_sym = Matrix(J)
print('Матрица Якоби:')
display(J_sym)

Матрица Якоби:


Matrix([
[-Q1**(n/(2*n + 3))*V1**(1/(2*n + 3))*n*(E1**(2*n + 2)*K*h1**(-3*n - 3)*(n + 3)*(2*n + 1)**n/(2**(2*n)*pi*n**n*phi**n))**(1/(2*n + 3))/(Q1*(2*n + 3)) - 8*Q1*rho_s*sign(Q1)/(pi**2*C_d1**2*d_p1**4*n_p1**2) - 8*rho_s*Abs(Q1)/(pi**2*C_d1**2*d_p1**4*n_p1**2),                                                                                                                                                                                                                                                            0,                                                                                                                                                                                                                                                            0, 1],
[                                                                                                                                                                                                                               

In [9]:
# числовые значения известных параметров задачи
# в массивах параметров трещин сдвиг индексов на 1, т.к. нумерация в массивах начинается с нуля
# например, чтобы изменить параметр для 1-ой трещины необходимо обратиться к нулевому элементу массива

parameter_values = {
    Q_0: 800 / 86400,  # расход на забое, м^3/с
    K: 0.00071,  # первый реологический параметр жидкости (смеси), Па*с
    n: 1,  # второй реологический параметр жидкости (смеси)
    phi: 0.3,  # безразмерный геометрический параметр трещины
    rho_s: 1400  # плотность жидкости (смеси), кг/м^3
}

for i in range(frac_count):
    parameter_values[V[i]] = 20  # объёмы трещин, м^3
    parameter_values[E[i]] = 1e10  # модули плоской деформации трещин, Па
    parameter_values[h[i]] = 2  # мощности продуктивной зоны, м
    parameter_values[n_p[i]] = 16  # количества перфораций
    parameter_values[d_p[i]] = 0.02  # диаметры перфораций, м
    parameter_values[C_d[i]] = 0.7  # безразмерные коэффициенты эррозии
    parameter_values[R[i]] = 0.08  # радиусы участков трубы между i-ой и (i-1)-ой трещинами, м
    parameter_values[L[i]] = 100  # длины участков трубы между i-ой и (i-1)-ой трещинами, м
    parameter_values[sigma_min[i]] = 1e7  # давления закрытия трещин, Па

# изменение числовых значений параметров трещин (если необходимо задать отдельно для каждой трещины)
# parameter_values[L[2]] = 1000000

In [10]:
# начальные приближения числовых значений неизвестных параметров задачи
solution_values = {}
for i in range(frac_count):
    solution_values[Q[i]] = N(Q_0.subs(parameter_values)/frac_count)  # расходы на каждой из трещин (начальное приближение)
solution_values[p_0] = N(sigma_min[0].subs(parameter_values))  # забойное давление (начальное приближение)
#solution_values[Q[0]] = 20 / 86400
print(f'Начальное приближение:\n{solution_values}')

Начальное приближение:
{Q1: 0.00308641975308642, Q2: 0.00308641975308642, Q3: 0.00308641975308642, p0: 10000000.0000000}


In [11]:
# вектор решения
solution_vector = np.array(list(N(Matrix(Q).subs(solution_values)))+[N(p_0.subs(solution_values))], dtype='float64').reshape(frac_count+1,1)
# предыдущий вектор решения
last_solution_vector = solution_vector - 100

# реализация метода Ньютона
while np.linalg.norm(solution_vector - last_solution_vector) > 1e-4:
    # печатаем текущее приближение к решению
    print(solution_values)
    # обновляем предыдущий вектор решения
    last_solution_vector = solution_vector.copy()
    # матрица Якоби
    J = np.array(N(J_sym.subs({**parameter_values, **solution_values})), dtype='float64')
    # вектор остатков
    residual_vector = np.array(N(Matrix(all_eqns).subs({**parameter_values, **solution_values})), dtype='float64')
    # новый вектор решения
    solution_vector -= np.matmul(np.linalg.inv(J), residual_vector)
    # обновление значений текущего вектора решения в словаре solution_values
    for i in range(frac_count):
        solution_values[Q[i]] = solution_vector.reshape(frac_count+1,)[i]
    solution_values[p_0] = solution_vector.reshape(frac_count+1,)[-1]
else:
    print(f'Итоговое решение:\n{solution_values}')

{Q1: 0.00308641975308642, Q2: 0.00308641975308642, Q3: 0.00308641975308642, p0: 10000000.0000000}
{Q1: 0.0030864672388794595, Q2: 0.0030864102558752103, Q3: 0.0030863817645045907, p0: 17374255.207987502}
Итоговое решение:
{Q1: 0.003086467239007829, Q2: 0.0030864102557234987, Q3: 0.003086381764527931, p0: 17374255.20790938}


In [12]:
for i in range(frac_count):
    print(f'Расход на {i+1}-ой трещине: Q{i+1} = {solution_values[Q[i]]*86400} м^3/сут')
print(f'Забойное давление: p0 = {solution_values[p_0] / 1e6} МПа')

Расход на 1-ой трещине: Q1 = 266.67076945027645 м^3/сут
Расход на 2-ой трещине: Q2 = 266.6658460945103 м^3/сут
Расход на 3-ой трещине: Q3 = 266.66338445521325 м^3/сут
Забойное давление: p0 = 17.37425520790938 МПа
